In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [34]:
import numpy as np
import pandas as pd

def rank_column(df, column, ascending=False):
    _df = df.copy()
    col_name = f"rank_{column}"
    if col_name not in _df.columns:
        _df[col_name] = np.nan
    _df.loc[_df[col_name].isna(), col_name] = _df.groupby("start_time")[column].rank(
        ascending=ascending
    )
    return _df    

def prepare_data(data: pd.DataFrame):
    """
    Prepare the data for the strategy
    """
    data = data.copy()
    window_size = 7 * 24 * 4  # Adjust this based on your exact data frequency
    prep_col = [
        "pct_change",
        "rolling_accumulated_pct_change",
        "rolling_variance_pct_change",
        "rank",
    ]
    for col in prep_col:
        if col not in data.columns:
            data[col] = np.nan
    ### pct chage
    data.loc[data["pct_change"].isna(), "pct_change"] = data.groupby(level=0)[
        "close"
    ].pct_change()
    ### rolling_acc_pct_change
    data.loc[
        data["rolling_accumulated_pct_change"].isna(),
        "rolling_accumulated_pct_change",
    ] = data.groupby(level=0)["pct_change"].transform(
        lambda x: (
            x.rolling(window=window_size, min_periods=1).apply(
                lambda y: np.prod(1 + y / 100)
            )
            - 1
        )
        * 100
    )
    data.loc[
        data["rolling_variance_pct_change"].isna(), "rolling_variance_pct_change"
    ] = data.groupby(level=0)["pct_change"].transform(
        lambda x: x.rolling(window=window_size, min_periods=1).var()
    )
    data = rank_column(data, "rolling_accumulated_pct_change", ascending=False)
    data = rank_column(data, "rolling_variance_pct_change", ascending=True)
    data.loc[data["rank"].isna(), "rank"] = (
        data["rank_rolling_accumulated_pct_change"]
        + data["rank_rolling_variance_pct_change"]
    ) / 2
    return data

In [35]:
from settings import DATA_DIR

df = pd.read_csv(DATA_DIR / "all.csv", parse_dates=[1, 2])
df.set_index(["symbol", "start_time"], inplace=True)
df.head()

end_time interval  number_of_trades  \
symbol start_time                                                           
BTC    2024-03-12 00:00:00 2024-03-12 00:14:59      15m               740   
       2024-03-12 00:15:00 2024-03-12 00:29:59      15m               544   
       2024-03-12 00:30:00 2024-03-12 00:44:59      15m               441   
       2024-03-12 00:45:00 2024-03-12 00:59:59      15m               666   
       2024-03-12 01:00:00 2024-03-12 01:14:59      15m               735   

                              close     high      low     open    volume  \
symbol start_time                                                          
BTC    2024-03-12 00:00:00  72253.0  72389.0  72129.0  72155.0  73.59512   
       2024-03-12 00:15:00  72189.0  72360.0  72177.0  72254.0  33.93085   
       2024-03-12 00:30:00  72351.0  72411.0  72182.0  72188.0  38.35377   
       2024-03-12 00:45:00  72077.0  72351.0  72063.0  72350.0  39.48791   
       2024-03-12 01:00:00  72329.0  72360.0  71970.0  72076.0  74.58844   

                                      tic            toc  
symbol start_time                                         
BTC    2024-03-12 00:00:00  1710201600000  1710202499999  
       2024-03-12 00:15:00  1710202500000  1710203399999  
       2024-03-12 00:30:00  1710203400000  1710204299999  
       2024-03-12 00:45:00  1710204300000  1710205199999  
       2024-03-12 01:00:00  1710205200000  1710206099999

In [36]:
ranked_df = prepare_data(df)


In [37]:
ranked_df.head()

end_time interval  number_of_trades  \
symbol start_time                                                           
BTC    2024-03-12 00:00:00 2024-03-12 00:14:59      15m               740   
       2024-03-12 00:15:00 2024-03-12 00:29:59      15m               544   
       2024-03-12 00:30:00 2024-03-12 00:44:59      15m               441   
       2024-03-12 00:45:00 2024-03-12 00:59:59      15m               666   
       2024-03-12 01:00:00 2024-03-12 01:14:59      15m               735   

                              close     high      low     open    volume  \
symbol start_time                                                          
BTC    2024-03-12 00:00:00  72253.0  72389.0  72129.0  72155.0  73.59512   
       2024-03-12 00:15:00  72189.0  72360.0  72177.0  72254.0  33.93085   
       2024-03-12 00:30:00  72351.0  72411.0  72182.0  72188.0  38.35377   
       2024-03-12 00:45:00  72077.0  72351.0  72063.0  72350.0  39.48791   
       2024-03-12 01:00:00  72329.0  72360.0  71970.0  72076.0  74.58844   

                                      tic            toc  pct_change  \
symbol start_time                                                      
BTC    2024-03-12 00:00:00  1710201600000  1710202499999         NaN   
       2024-03-12 00:15:00  1710202500000  1710203399999   -0.000886   
       2024-03-12 00:30:00  1710203400000  1710204299999    0.002244   
       2024-03-12 00:45:00  1710204300000  1710205199999   -0.003787   
       2024-03-12 01:00:00  1710205200000  1710206099999    0.003496   

                            rolling_accumulated_pct_change  \
symbol start_time                                            
BTC    2024-03-12 00:00:00                             NaN   
       2024-03-12 00:15:00                       -0.000886   
       2024-03-12 00:30:00                        0.001358   
       2024-03-12 00:45:00                       -0.002429   
       2024-03-12 01:00:00                        0.001067   

                            rolling_variance_pct_change  rank  \
symbol start_time                                               
BTC    2024-03-12 00:00:00                          NaN   NaN   
       2024-03-12 00:15:00                          NaN   NaN   
       2024-03-12 00:30:00                     0.000005  26.0   
       2024-03-12 00:45:00                     0.000009  17.5   
       2024-03-12 01:00:00                     0.000011  16.0   

                            rank_rolling_accumulated_pct_change  \
symbol start_time                                                 
BTC    2024-03-12 00:00:00                                  NaN   
       2024-03-12 00:15:00                                 44.0   
       2024-03-12 00:30:00                                 33.0   
       2024-03-12 00:45:00                                 25.0   
       2024-03-12 01:00:00                                 21.0   

                            rank_rolling_variance_pct_change  
symbol start_time                                             
BTC    2024-03-12 00:00:00                               NaN  
       2024-03-12 00:15:00                               NaN  
       2024-03-12 00:30:00                              19.0  
       2024-03-12 00:45:00                              10.0  
       2024-03-12 01:00:00                              11.0

In [38]:
def get_non_volatile_coins(df):
    # Group by the 'symbol' level of the MultiIndex and get the index of the max 'start_time' for each 'symbol'
    idx = df.groupby(level='symbol')['end_time'].idxmax()

    # Select the rows with the latest 'end_time' for each 'symbol'
    latest_df = df.loc[idx]

    # Assuming 'rank' is a column in your DataFrame, sort these rows by 'rank' in ascending order
    sorted_latest_df = latest_df.sort_values(by='rank_rolling_variance_pct_change', ascending=True)

    print("dropped:", sorted_latest_df.index.get_level_values('symbol').tolist()[-10:])
    return sorted_latest_df.index.get_level_values('symbol').tolist()[:-10]

good_coins = get_non_volatile_coins(ranked_df)
len(good_coins)

dropped: ['ONDO', 'TAO', 'ALT', 'FTM', 'NEO', 'ORDI', 'JTO', 'WIF', 'PENDLE', 'W']


48

In [39]:
ranked_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 167098 entries, ('BTC', Timestamp('2024-03-12 00:00:00')) to ('W', Timestamp('2024-04-11 00:00:00'))
Data columns (total 16 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   end_time                             167098 non-null  datetime64[ns]
 1   interval                             167098 non-null  object        
 2   number_of_trades                     167098 non-null  int64         
 3   close                                167098 non-null  float64       
 4   high                                 167098 non-null  float64       
 5   low                                  167098 non-null  float64       
 6   open                                 167098 non-null  float64       
 7   volume                               167098 non-null  float64       
 8   tic                                  167098 non-null  int64         
 9

In [43]:
cols = [
    "pct_change",
    "rolling_accumulated_pct_change",
    "rank_rolling_accumulated_pct_change",
    "rolling_variance_pct_change",
    "rank_rolling_variance_pct_change",
    "rank",
]
non_volatile_reranked_df = ranked_df.loc[good_coins].drop(cols, axis=1)
non_volatile_reranked_df = prepare_data(non_volatile_reranked_df)

In [44]:
non_volatile_reranked_df.head()

end_time interval  number_of_trades  \
symbol start_time                                                           
TRX    2024-03-12 00:00:00 2024-03-12 00:14:59      15m                26   
       2024-03-12 00:15:00 2024-03-12 00:29:59      15m                20   
       2024-03-12 00:30:00 2024-03-12 00:44:59      15m                21   
       2024-03-12 00:45:00 2024-03-12 00:59:59      15m                32   
       2024-03-12 01:00:00 2024-03-12 01:14:59      15m                59   

                              close     high      low     open    volume  \
symbol start_time                                                          
TRX    2024-03-12 00:00:00  0.13390  0.13394  0.13384  0.13385  208789.0   
       2024-03-12 00:15:00  0.13395  0.13397  0.13390  0.13390    3276.0   
       2024-03-12 00:30:00  0.13401  0.13404  0.13394  0.13394    3400.0   
       2024-03-12 00:45:00  0.13388  0.13402  0.13388  0.13402  134846.0   
       2024-03-12 01:00:00  0.13391  0.13393  0.13379  0.13388  479736.0   

                                      tic            toc  pct_change  \
symbol start_time                                                      
TRX    2024-03-12 00:00:00  1710201600000  1710202499999         NaN   
       2024-03-12 00:15:00  1710202500000  1710203399999    0.000373   
       2024-03-12 00:30:00  1710203400000  1710204299999    0.000448   
       2024-03-12 00:45:00  1710204300000  1710205199999   -0.000970   
       2024-03-12 01:00:00  1710205200000  1710206099999    0.000224   

                            rolling_accumulated_pct_change  \
symbol start_time                                            
TRX    2024-03-12 00:00:00                             NaN   
       2024-03-12 00:15:00                        0.000373   
       2024-03-12 00:30:00                        0.000821   
       2024-03-12 00:45:00                       -0.000149   
       2024-03-12 01:00:00                        0.000075   

                            rolling_variance_pct_change  rank  \
symbol start_time                                               
TRX    2024-03-12 00:00:00                          NaN   NaN   
       2024-03-12 00:15:00                          NaN   NaN   
       2024-03-12 00:30:00                 2.776268e-09  16.0   
       2024-03-12 00:45:00                 6.368760e-07   8.5   
       2024-03-12 01:00:00                 4.433064e-07  10.0   

                            rank_rolling_accumulated_pct_change  \
symbol start_time                                                 
TRX    2024-03-12 00:00:00                                  NaN   
       2024-03-12 00:15:00                                 31.0   
       2024-03-12 00:30:00                                 30.0   
       2024-03-12 00:45:00                                 16.0   
       2024-03-12 01:00:00                                 19.0   

                            rank_rolling_variance_pct_change  
symbol start_time                                             
TRX    2024-03-12 00:00:00                               NaN  
       2024-03-12 00:15:00                               NaN  
       2024-03-12 00:30:00                               2.0  
       2024-03-12 00:45:00                               1.0  
       2024-03-12 01:00:00                               1.0

In [45]:
non_volatile_reranked_df.index.get_level_values('symbol').nunique()

48

In [46]:
non_volatile_reranked_df.to_csv(DATA_DIR / "ranked.csv")

In [21]:
non_volatile_reranked_df.head()

end_time interval  number_of_trades  \
symbol start_time                                                           
TRX    2024-03-12 00:00:00 2024-03-12 00:14:59      15m                26   
       2024-03-12 00:15:00 2024-03-12 00:29:59      15m                20   
       2024-03-12 00:30:00 2024-03-12 00:44:59      15m                21   
       2024-03-12 00:45:00 2024-03-12 00:59:59      15m                32   
       2024-03-12 01:00:00 2024-03-12 01:14:59      15m                59   

                              close     high      low     open    volume  \
symbol start_time                                                          
TRX    2024-03-12 00:00:00  0.13390  0.13394  0.13384  0.13385  208789.0   
       2024-03-12 00:15:00  0.13395  0.13397  0.13390  0.13390    3276.0   
       2024-03-12 00:30:00  0.13401  0.13404  0.13394  0.13394    3400.0   
       2024-03-12 00:45:00  0.13388  0.13402  0.13388  0.13402  134846.0   
       2024-03-12 01:00:00  0.13391  0.13393  0.13379  0.13388  479736.0   

                                      tic            toc  pct_change  \
symbol start_time                                                      
TRX    2024-03-12 00:00:00  1710201600000  1710202499999         NaN   
       2024-03-12 00:15:00  1710202500000  1710203399999    0.000373   
       2024-03-12 00:30:00  1710203400000  1710204299999    0.000448   
       2024-03-12 00:45:00  1710204300000  1710205199999   -0.000970   
       2024-03-12 01:00:00  1710205200000  1710206099999    0.000224   

                            rolling_accumulated_pct_change  \
symbol start_time                                            
TRX    2024-03-12 00:00:00                             NaN   
       2024-03-12 00:15:00                        0.000373   
       2024-03-12 00:30:00                        0.000821   
       2024-03-12 00:45:00                       -0.000149   
       2024-03-12 01:00:00                        0.000075   

                            rolling_variance_pct_change  rank  \
symbol start_time                                               
TRX    2024-03-12 00:00:00                          NaN   NaN   
       2024-03-12 00:15:00                          NaN   NaN   
       2024-03-12 00:30:00                 2.776268e-09  16.0   
       2024-03-12 00:45:00                 6.368760e-07   8.5   
       2024-03-12 01:00:00                 4.433064e-07  10.0   

                            rank_rolling_accumulated_pct_change  \
symbol start_time                                                 
TRX    2024-03-12 00:00:00                                  NaN   
       2024-03-12 00:15:00                                 31.0   
       2024-03-12 00:30:00                                 30.0   
       2024-03-12 00:45:00                                 16.0   
       2024-03-12 01:00:00                                 19.0   

                            rank_rolling_variance_pct_change  
symbol start_time                                             
TRX    2024-03-12 00:00:00                               NaN  
       2024-03-12 00:15:00                               NaN  
       2024-03-12 00:30:00                               2.0  
       2024-03-12 00:45:00                               1.0  
       2024-03-12 01:00:00                               1.0

In [47]:
non_volatile_reranked_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 138288 entries, ('TRX', Timestamp('2024-03-12 00:00:00')) to ('JUP', Timestamp('2024-04-11 00:00:00'))
Data columns (total 16 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   end_time                             138288 non-null  datetime64[ns]
 1   interval                             138288 non-null  object        
 2   number_of_trades                     138288 non-null  int64         
 3   close                                138288 non-null  float64       
 4   high                                 138288 non-null  float64       
 5   low                                  138288 non-null  float64       
 6   open                                 138288 non-null  float64       
 7   volume                               138288 non-null  float64       
 8   tic                                  138288 non-null  int64         


In [48]:
# Group by the 'symbol' level of the MultiIndex and get the index of the max 'start_time' for each 'symbol'
idx = non_volatile_reranked_df.groupby(level='symbol')['end_time'].idxmax()

# Select the rows with the latest 'end_time' for each 'symbol'
latest_df = non_volatile_reranked_df.loc[idx]

# Assuming 'rank' is a column in your DataFrame, sort these rows by 'rank' in ascending order
sorted_latest_df = latest_df.sort_values(by='rank', ascending=True)


In [49]:
sorted_latest_df.loc[:, [ "end_time", "rank"]]

,,end_time,rank
symbol,start_time,,
BNB,2024-04-11,2024-04-11 00:14:59,4.5
BTC,2024-04-11,2024-04-11 00:14:59,5.5
ETH,2024-04-11,2024-04-11 00:14:59,6.5
XRP,2024-04-11,2024-04-11 00:14:59,9.5
TRX,2024-04-11,2024-04-11 00:14:59,10.5
ADA,2024-04-11,2024-04-11 00:14:59,13.0
ATOM,2024-04-11,2024-04-11 00:14:59,14.0
kSHIB,2024-04-11,2024-04-11 00:14:59,15.0
DYDX,2024-04-11,2024-04-11 00:14:59,16.5


In [50]:
sorted_latest_df = sorted_latest_df.loc[:, [ "end_time", "rank"]]
sorted_latest_df

,,end_time,rank
symbol,start_time,,
BNB,2024-04-11,2024-04-11 00:14:59,4.5
BTC,2024-04-11,2024-04-11 00:14:59,5.5
ETH,2024-04-11,2024-04-11 00:14:59,6.5
XRP,2024-04-11,2024-04-11 00:14:59,9.5
TRX,2024-04-11,2024-04-11 00:14:59,10.5
ADA,2024-04-11,2024-04-11 00:14:59,13.0
ATOM,2024-04-11,2024-04-11 00:14:59,14.0
kSHIB,2024-04-11,2024-04-11 00:14:59,15.0
DYDX,2024-04-11,2024-04-11 00:14:59,16.5


In [51]:
sorted_latest_df.to_csv(DATA_DIR / "sorted_latest.csv")